In [47]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from PIL import Image

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
# from imutils import contours
import shutil

%matplotlib inline

TRAINING_PATH = 'push/'
BATCH_SIZE = 16
IMG_SHAPE = (128, 128)

In [48]:
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)
    

In [49]:
TRAINING_PATH = 'push/'
def test_code(root_dir):
    root_dir = root_dir
    categories = sorted(os.listdir(root_dir))
    cat2idx = dict(zip(categories, range(len(categories))))
    idx2cat = dict(zip(cat2idx.values(), cat2idx.keys()))
    files = []
    cat_mapping = {}
    for (dirpath, dirnames, filenames) in os.walk(root_dir):
        print(dirpath, dirnames, filenames)
        for f in filenames:
            if f.endswith('.png'):
                o = {}
                o['img_path'] = dirpath + '/' + f
                o['category'] = cat2idx[dirpath[dirpath.find('/')+1:]]
                cat_mapping[o['category']] = dirpath.split('/')[-1]
                files.append(o)
    print(files)
    print(f'Categorie mapping: {cat_mapping}')

test_code(TRAINING_PATH)

push/ ['pushup', 'squat'] []
push/pushup [] ['100.png', '1000.png', '1005.png', '1010.png', '1015.png', '1020.png', '1025.png', '1030.png', '1035.png', '1040.png', '1045.png', '105.png', '1050.png', '1055.png', '1060.png', '1065.png', '1070.png', '1075.png', '1080.png', '1085.png', '1090.png', '1095.png', '110.png', '1100.png', '1105.png', '1110.png', '1115.png', '1120.png', '1125.png', '1130.png', '1135.png', '1140.png', '1145.png', '115.png', '1150.png', '1155.png', '1160.png', '1165.png', '1170.png', '1175.png', '1180.png', '1185.png', '1190.png', '1195.png', '120.png', '1200.png', '1205.png', '1210.png', '1215.png', '1220.png', '1225.png', '1230.png', '1235.png', '1240.png', '1245.png', '125.png', '1250.png', '1255.png', '1260.png', '1265.png', '1270.png', '1275.png', '1280.png', '1285.png', '1290.png', '1295.png', '130.png', '1300.png', '1305.png', '1310.png', '1315.png', '1320.png', '1325.png', '1330.png', '1335.png', '1340.png', '1345.png', '135.png', '1350.png', '1355.png', '13

In [50]:
# define basic image transforms for preprocessing
transform = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(0.2),
    RandomShift(0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5,), std = (0.5, ))
])

class Dataset(torch.utils.data.Dataset):
    '''
    Custom Dataset object for the CDiscount competition
        Parameters:
            root_dir - directory including category folders with images

        Example:
        images/
            1000001859/
                26_0.jpg
                26_1.jpg
                ...
            1000004141/
                ...
            ...
    '''
    
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.categories = sorted(os.listdir(root_dir))
        self.cat2idx = dict(zip(self.categories, range(len(self.categories))))
        self.idx2cat = dict(zip(self.cat2idx.values(), self.cat2idx.keys()))
        self.files = []
        cat_mapping = {}
        for (dirpath, dirnames, filenames) in os.walk(self.root_dir):
            for f in filenames:
                if f.endswith('.png'):
                    o = {}
                    o['img_path'] = dirpath + '/' + f
                    o['category'] = self.cat2idx[dirpath[dirpath.find('/')+1:]]
                    cat_mapping[o['category']] = dirpath.split('/')[-1]
                    self.files.append(o)
        self.transform = transform
        print(f'Categorie mapping: {cat_mapping}')
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_path = self.files[idx]['img_path']
        category = self.files[idx]['category']
        image = cv2.imread(img_path)
        image = cv2.resize(image, IMG_SHAPE)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
            
        return {'image': image, 'category': category}


# create instance of data class and pytorch dataloader
dataSet = Dataset(TRAINING_PATH, transform=transform)
dataloader = torch.utils.data.DataLoader(dataSet, batch_size=BATCH_SIZE, shuffle=True)

Categorie mapping: {0: 'pushup', 1: 'squat'}


In [51]:
from Net import Net


net = Net()
print('######### Network created #########')
print('Architecture:\n', net)

### Train
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

print('Started Training!')
net.train()
for epoch in range(10):
    running_loss = 0.0
    examples = 0
    for i, data in enumerate(dataloader, 0):
        # Get the inputs
        inputs, labels = data['image'], data['category']
        
        # Wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.data
        examples += BATCH_SIZE
    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / examples))

print('Finished Training!')

######### Network created #########
Architecture:
 Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=46656, out_features=512, bias=True)
  (bnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (bnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (bnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=64, out_features=3, bias=True)
)
Started Training!
[1,    83] loss: 0.010
[2,    8

In [12]:
import re
'''
From: https://stackoverflow.com/questions/4623446/how-do-you-sort-files-numerically
'''
def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

In [17]:
from IPython.display import clear_output
net.eval()

transform = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5,), std = (0.5, ))
])
PATH = 'classification/IMG_8266/'
classes = []
for frame in sorted(os.listdir(PATH), key = alphanum_key):
    image = cv2.imread(PATH + frame)
    image = cv2.resize(image, IMG_SHAPE)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    x = transform(image).reshape(1, 3, 128, 128)
    classes.append(np.argmax(net(x).detach().numpy()))

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-17-6f4df46a2ae5>, line 10)

In [37]:
PATH = 'C://Users//29433//Documents//GitHub//workout-movement-counting//test_code//OptFlowtest'
# image = cv2.imread(PATH + '//10.png')
# print(image.shape)
# image = cv2.resize(image, IMG_SHAPE)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# x = transform(image).reshape(1, 3, 128, 128)
# print(np.argmax(net(x).detach().numpy()))
for frame in sorted(os.listdir(PATH)):
    image = cv2.imread(PATH + '//' + frame)
#     print(image.shape)
    image = cv2.resize(image, IMG_SHAPE)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    x = transform(image).reshape(1, 3, 128, 128)
    classes.append(np.argmax(net(x).detach().numpy()))

In [38]:
def moveCount(labels):
    k = 0
    t = 0
    for i in range(1, len(labels)):
        if labels[i-1] == 0 and labels[i] == 2:
            k+=1
    for i in range(2, len(labels)-1):
        if labels[i-2] == 0 and labels[i-1] == 1 and labels[i] == 2 and labels[i+1] == 2:
            t+=1
    return k + t

In [39]:
moveCount(classes)

26

In [30]:
# save torch model for further prediction
torch.save(net.state_dict(), 'model.pt')